<a href="https://colab.research.google.com/github/junkuna/Machine-Learning-Training/blob/main/5_3_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# We need best model based on previous Decision Tree model using Random Forest
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

## train the model with RandomForest
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print('The scores of DecisionTree through RandomForest are')
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

### check the importance of feature
### RandomForest support every features to get trained in random
rf.fit(train_input, train_target)
print(rf.feature_importances_)
print('If we compare with previous features, the pH increased and sugar decreased')

#### we can use left over of bootstrap sample to score the DecisionTree
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

### Leverage ExtraTree to see the difference with RandomForest
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print('The scores of DecisionTrees using ExtraTrees are ')
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

et.fit(train_input, train_target)
print('The feature importances through ExtraTrees are')
print(et.feature_importances_)

## Train the model with GradientBoosting
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                                random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print('The model scores and feature importances with GradientBoosting are')
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb.fit(train_input, train_target)
print(gb.feature_importances_)

## Use Histogram-Based Gradient Boosting that is better than Gradient Boosting
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print('The model scores with Histogram Gradient Boosting are')
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

## check the feature importance with mixing features by permutation importance
### check the score of feature in train sets
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                               n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

### check the score of feature in test sets
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

## check final feature importance
hgb.score(test_input, test_target)

## Check the score through XGBoost library
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

## check the score through LightGBM
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

The scores of DecisionTree through RandomForest are
0.9973541965122431 0.8905151032797809
[0.23167441 0.50039841 0.26792718]
If we compare with previous features, the pH increased and sugar decreased
0.8934000384837406
The scores of DecisionTrees using ExtraTrees are 
0.9974503966084433 0.8887848893166506
The feature importances through ExtraTrees are
[0.20183568 0.52242907 0.27573525]
0.8881086892152563 0.8720430147331015
The model scores and feature importances with GradientBoosting are
0.9464595437171814 0.8780082549788999
[0.15872278 0.68010884 0.16116839]
The model scores with Histogram Gradient Boosting are
0.9321723946453317 0.8801241948619236
[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]
0.9558403027491312 0.8782000074035686
0.935828414851749 0.8801251203079884
